In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point
import streetmapper

In [3]:
street_segments = gpd.read_file("../data/processed/street_segments.geojson")
blocks = gpd.read_file("../data/processed/blocks.geojson")
buildings = gpd.read_file("../data/processed/buildings.geojson")
blockfaces = gpd.read_file("../data/processed/blockfaces.geojson")

In [4]:
trash = pd.read_excel("../data/one-month-data.xlsx")
trash = gpd.GeoDataFrame(trash, 
                         geometry=trash.apply(lambda srs: Point(srs['long'], srs['lat']),axis='columns'))

In [5]:
selection_area = Polygon(
    ((-122.422836714479, 37.7849452051136), 
     (-122.419309755901, 37.7849452051136), 
     (-122.419309755901, 37.7998665826739), 
     (-122.422836714479, 37.7998665826739), 
     (-122.422836714479, 37.7849452051136))
).buffer(0.001)


blocks_of_interest = streetmapper.pipeline.select_area_of_interest(blocks, selection_area)
frontages_of_interest = streetmapper.pipeline.calculate_frontages(blocks_of_interest, 
                                                                street_segments, blockfaces, buildings)
trash_of_interest = streetmapper.pipeline.select_area_of_interest(trash, selection_area)

100%|██████████| 71/71 [01:14<00:00,  1.13s/it]


In [6]:
f_index = streetmapper.pipeline.create_index(frontages_of_interest)
trash_frontage_mapping = streetmapper.pipeline.assign_points_to_frontages(trash_of_interest, 
                                                                          frontages_of_interest, f_index)

In [7]:
augmented_trash = trash_of_interest.assign(sf16_BldgID_n=trash_frontage_mapping.values)

In [8]:
augmented_trash.head()

,itemsTagged,likes,street,type,time,userPrimaryCommunityName,userCity,userCityDistrict,userState,userZipCode,lat,long,totalNumberOfItemsTagged,pickedUp,geometry,sf16_BldgID_n
0,1,0,Filbert St,tobacco,"9/18/2018, 12:45:07 AM",Russian Hill,San Francisco,San Francisco County,California,94109,37.799602,-122.422209,1,True,POINT (-122.422208928493 37.7996024350634),201006.0022929_0
1,1,0,Filbert St,other,"9/18/2018, 12:45:05 AM",Russian Hill,San Francisco,San Francisco County,California,94109,37.799808,-122.422168,1,True,POINT (-122.422168068763 37.7998081601673),201006.0014843_1
2,1,0,Filbert St,other,"9/18/2018, 12:45:04 AM",Russian Hill,San Francisco,San Francisco County,California,94109,37.799744,-122.422153,1,True,POINT (-122.422152552981 37.7997437724664),201006.0022929_0
3,1,0,Filbert St,other,"9/18/2018, 12:45:03 AM",Russian Hill,San Francisco,San Francisco County,California,94109,37.799807,-122.422091,1,True,POINT (-122.422090792739 37.7998066128486),201006.0020953_0
4,1,0,Polk St,tobacco,"9/18/2018, 12:42:13 AM",Russian Hill,San Francisco,San Francisco County,California,94109,37.799505,-122.422302,1,True,POINT (-122.422302477379 37.799504567795),201006.0031962_0


In [9]:
(augmented_trash
     .assign(pickedUp=augmented_trash.pickedUp.astype(int))
     .to_file('../data/processed/trash-polk.geojson', driver='GeoJSON'))

DriverIOError: GeoJSON driver doesn't support creating a layer on a read-only datasource

In [ ]:
frontages_of_interest.to_file("../data/processed/frontages-polk.geojson", driver='GeoJSON')